# Painel Fotovoltaico
### Autora: Luana Dias Costa Lucas <br> Empresa : WP SOLUÇÕES INDUSTRIAIS INTELIGENTES LTDA


## RELATÓRIO PREVISÃO

&emsp;Para adquir os dados climaticos para os proximos dias foi feita uma chamada de API fornecida pelo site https://openweathermap.org/appid

&emsp;Idealmente, para a comparação os dados da previsão do tempo e adquirir a previsao de geração de energia, seria necessário comparar os dados de radiação solar, porem essa informação faz parte de um pacote pago fornecido pelo site comentado anteriormente. Por isso foi escolhido comparar a hora do dia e a temperatura da previsão com o histórico de dados para tentar realizar essa previsão de energia.

&emsp;Por mais que não seja correto, funcionou como um meio de demonstrar o funcionamento do codigo e a implementação da lógica

In [3]:
#Bibliotecas
from ipynb.fs.full.Helper import *
from datetime import date, datetime, timedelta
from fpdf import FPDF
import os
import pandas as pd
import numpy as np

#generationData = pd.read_csv('archive/Plant_1_Generation_Data.csv')
#weatherData = pd.read_csv('archive/Plant_1_Weather_Sensor_Data.csv')
#plant = '1'

In [ ]:
# Escolha da central solar
plant = whichPlant()
        
if plant == '1':
    path = 'archive/Plant_1/'
else:
    path = 'archive/Plant_2/'
generationData = pd.read_csv(path + 'Generation_Data.csv')
weatherData = pd.read_csv(path + 'Weather_Sensor_Data.csv')

In [5]:
# Consolidar os dados das duas bases (geracao de energia e informacoes do tempo) em uma unica tabela
generationData_sort = generationData.drop(columns = ['PLANT_ID', 'SOURCE_KEY', 'DAILY_YIELD', 'TOTAL_YIELD']).reset_index().drop(columns = ['index'])
x = len(generationData_sort)

time = []
temp = []
for i in range(x):
    date_time = datetime.strptime(generationData_sort['DATE_TIME'][i], '%d-%m-%Y %H:%M')
    time.append(date_time.strftime("%H:%M"))
    if str(date_time) in weatherData['DATE_TIME'].values:
        tempAmb = weatherData[weatherData['DATE_TIME']==str(date_time)].iloc[0]['AMBIENT_TEMPERATURE']
        temp.append(tempAmb)
    else:
        temp.append('NaN')

generationData_sort['AMBIENT_TEMPERATURE'] = temp
generationData_sort['TIME'] = time
generationData_Weather = generationData_sort


In [6]:
# Chamada da API para obter a previsao do tempo para os proximos 5 dias, em um intervalo de 3h
dataForecast = forecast()
x = len(dataForecast)

dc_power = []	
ac_power = []
daily_yield = [] 
data = date.today() - timedelta(days = 1)

# Tentativa de prever a geracao de energia considerando apenas a temp e a hora do dia
for i in range(x):
    date_time = datetime.strptime(dataForecast['data'][i], '%Y-%m-%d %H:%M:%S')
    time = date_time.strftime("%H:%M")
    temp = dataForecast['temp'][i]
    if time in generationData_Weather['TIME'].values:
        sameTime = generationData_Weather[generationData_Weather['TIME'] == time]
        sameTemp = sameTime[sameTime['AMBIENT_TEMPERATURE'] >= float(temp)-1]
        sameTemp = sameTemp[sameTemp['AMBIENT_TEMPERATURE'] <= float(temp)+3]
        sameTemp_Sum = sameTemp.median(numeric_only=True)['DC_POWER']
        if np.isnan(sameTemp_Sum):
            dc_power.append('0.0')
            ac_power.append('0.0')
        else:
            dc_power.append(sameTemp_Sum)
            # Considerando que AC_Power é 90% de DC_Power
            ac_power.append(sameTemp_Sum*0.9)
    else:
        dc_power.append('0.0')
        ac_power.append('0.0')

dataForecast['DC_POWER (kW/15min)'] = dc_power
dataForecast['AC_POWER (kW/15min)'] = ac_power

for i in range(x):
    onlyData = datetime.strptime(dataForecast['data'][i], '%Y-%m-%d %H:%M:%S').date()
    if data != onlyData:
        data = onlyData
        daily_yield.append('0.0')
    else:
        media = (float(dataForecast['AC_POWER (kW/15min)'][i-1]) + float(dataForecast['AC_POWER (kW/15min)'][i]))/2
        daily_yield.append((media/4) + float(daily_yield[i-1]))

dataForecast['DAILY_YIELD (kW/h)'] = daily_yield

In [21]:
# Consolidação da previsão 
x = len(dataForecast)
dia = []
sumEnergyDaily = []
medTemp = []

lenStart = 0
for i in range(x-1):
    dataS = datetime.strptime(dataForecast['data'][lenStart], '%Y-%m-%d %H:%M:%S').date()
    dataF = datetime.strptime(dataForecast['data'][i+1], '%Y-%m-%d %H:%M:%S').date()
    if dataS != dataF:
        dia.append(dataS)
        medTemp.append(dataForecast[lenStart:i+1].median()['temp'])
        sumEnergyDaily.append(dataForecast.iloc[i]['DAILY_YIELD (kW/h)'])
        lenStart = i+1
dia.append(dataS)
medTemp.append(dataForecast[lenStart:i+2].median()['temp'])
sumEnergyDaily.append(float(dataForecast.iloc[i]['DAILY_YIELD (kW/h)']))    

previsao = {'Data': dia, 'Media_Temperatura': medTemp, 'Geracao_de_Energia_kW_h': sumEnergyDaily}
dfPrevisao = pd.DataFrame(previsao)

,Data,Media_Temperatura,Geracao_de_Energia_kW_h
0,2023-12-05,20.435,9.161719
1,2023-12-06,21.560,1546.629911
2,2023-12-07,22.020,2577.539732
3,2023-12-08,20.670,1475.276786
4,2023-12-09,19.485,496.904464
5,2023-12-10,19.100,0.000000


In [20]:
pdf = FPDF()
# Set left margin
pdf.set_left_margin(0)
pdf.add_page()
pdf.set_font('Arial', 'B', 14)

## Title
nInversores = len(generationData['SOURCE_KEY'].unique())
primeiroDia = str(dfPrevisao.iloc[0]['Data'])
ultimoDia = str(dfPrevisao.iloc[len(dfPrevisao)-1]['Data'])
pdf.multi_cell(0,10,'Previsão de geração de energia para cada um dos ' + str(nInversores) + ' inversores entre os dias ' + primeiroDia + ' e ' + ultimoDia, align='C')
pdf.ln(20)
pdf.set_font('Arial', '', 10)

dfPrevisao_pdf = dfPrevisao

spreadsheet_df_to_pdf(pdf, dfPrevisao_pdf, 3, 20, True)

dataHoje = date.today().strftime("%d%m%Y")

if not os.path.exists('Relatorios/Previsao/Plant' + plant):
   os.makedirs('Relatorios/Previsao/Plant' + plant)

pdf.output('Relatorios/Previsao/Plant' + plant +'/Previsao para o proximos 5 dias a partir de ' + dataHoje + '.pdf', 'F')

print('Relatório de previsão gerado com sucesso!')


''